# **THE OBJECTIVE**

Write a script which solves the task given in file 0a938d79.json in training directory. 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Accessing data

In the code block below, we learned

* Going to Data tab on right enables you to access competition data. Useful to know!
* Apparently, we want to use "Path" module from "pathlib" library to define "paths" that we can input instead of the raw strings representing the paths to the appropriate directory. 

All more or less from the Starter Notebook


In [ ]:
from pathlib import Path

for dirpath, dirname, files in os.walk('/kaggle/input/'):
    print(dirpath)
    print(dirname)
    
data_path = Path(r'/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'

print(training_path)

# **Working with .json files**
In the next block, we familiarize ourselves with objects in .json files comprising our data by addressing
1. How do we define a list of the file names in the directory of interest?
2. How do we print contents of .json file?
3. From (2) we see the contents of the given task file (any task file, for that matter) is a single dictionary. What are its keys?
4. From (2), we also see that each value of the dictionary (file) is a list of dictionaries. These dictionaries are input/output pairs so again, two-key dictionary one key for input one key for output. Can you "take out the data" so you don't have to keep opening the file? 
5. How do you plot whatever matrix/grid contained in a task? (We know "display()" function from Starter Notebook but we want nicer depiction. 


In [ ]:
# (1) Defining list of .json file names comprising training directory

training_tasks = sorted(os.listdir(training_path))

# (2) Printing contents of .json files in Python
# (3) Listing the keys of the dictionary comprising the task file
# (4) "Taking out the data" by defining labels for the dictionaries corresponding to i/o pairs. 

import json

with open(training_path / '0a938d79.json', 'r') as task:
    task_contents = json.load(task)
#(2)    print(task_contents)  
#(3)    print(list(task_contents.keys()))
#(4) Portion directly below
    demon_ex0 = task_contents['train'][0]
    demon_ex1 = task_contents['train'][1]
    demon_ex2 = task_contents['train'][2]
    demon_ex3 = task_contents['train'][3]
    test_pair = task_contents['test'][0]
    
#(5) Defining function which takes i/o matrix pair as input and outputs
# a grid with the coloring convention given in the ARC app  

from matplotlib import colors
import matplotlib.pyplot as plt 

def depiction(matrix):
    """
    A function which will depict the matrix a la the ARC html app
    """
    cmap = colors.ListedColormap(
            ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
             '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25']) # Ad hoc, from Starter

    norm = colors.Normalize(vmin=0, vmax=9) # Ad hoc, from Starter

    plt.matshow(matrix, cmap = cmap, norm = norm) 
    plt.grid('w')
    ax = plt.gca() 
    plt.tick_params(labelleft=False, labeltop=False) 
    ax.set_xticks(np.arange(-0.5, len(matrix[0]), 1));
    ax.set_yticks(np.arange(-0.5, len(matrix), 1));








# Scripting Solution

In the code block below, we start sketching the script which will successfully complete the task. By "successful completion" we mean:
* The script will simply take the test input matrix and output the correct test output
* The script's output production will be based on "functionality" we think will generalize to other ARC tasks (i.e., it won't simply be some trivial script where we print the solution having obtained it by hand)
* The script should be able to produce the correct output for every conveivable input of this "task type". That is, if we were to run the script on a hand-produced input that fits the perceived type, it should produce the correct output. 
* The script will choose output grid based on hand-solution. We will take into account grid dimension choice with other work


In [ ]:
# Defining repeated column fill and repeated row fill functions

def rep_row_fill(matrix):
    """
    Takes input grid/matrix, flushes grid accordingly 
    I.e. in accordance with how hand-solution dictates
    """
    N_colors = [] # list of number of distinct colors in each row
    color_indices = [] # indices of rows with non-black colors
    
    for i in range(len(matrix)):
        N_colors.append(len(np.unique(matrix[i])))                                            
    for i in range(len(N_colors)):
        if N_colors[i] == max(N_colors):
            color_indices.append(i)
                            
    delta = color_indices[1]-color_indices[0]
        
    # First row color flushes
    for i in range(color_indices[0], len(N_colors), 2*delta):
        for j in range(len(matrix[0])):
            matrix[i][j] = np.unique(matrix[color_indices[0]])[-1]                        
    # Second row color flushes
    for i in range(color_indices[1], len(N_colors), 2*delta):
        for j in range(len(matrix[0])):
            matrix[i][j] = np.unique(matrix[color_indices[1]])[-1]                     
    return matrix
                        
def rep_col_fill(matrix):
    """
    Takes input grid/matrix, flushes grid accordingly 
    I.e. in accordance with how hand-solution dictates
    """
    N_colors = [] # list of number of distinct colors in each column
    color_indices = [] # indices of columns with non-black colors
    col_form = np.array(matrix).T
    
    for j in range(len(col_form)):
        N_colors.append(len(np.unique(col_form[j])))                            
    for i in range(len(N_colors)):
        if N_colors[i] == max(N_colors):
            color_indices.append(i)
            
    delta = color_indices[1]-color_indices[0]
    
    # First column color flushes
    for j in range(color_indices[0], len(N_colors), 2*delta):
        for i in range(len(matrix)):
            matrix[i][j] = np.unique(col_form[color_indices[0]])[-1]
    # Second column color flushes
    for j in range(color_indices[1], len(N_colors), 2*delta):
        for i in range(len(matrix)):
            matrix[i][j] = np.unique(col_form[color_indices[1]])[-1] 
    return matrix


# Defining solution function

def output(in_matrix):
    if len(in_matrix) > len(in_matrix[0]): # Taller than wide
        rep_row_fill(in_matrix)
    elif len(in_matrix) < len(in_matrix[0]): # Wider than tall
        rep_col_fill(in_matrix)    
    return in_matrix

depiction(test_pair['input'])
depiction(test_pair['output'])
depiction(output(test_pair['input']))
    


